## [함수형 프로그래밍 HOWTO (4)](https://docs.python.org/ko/3/howto/functional.html)

<목차>
- functools 모듈
  - functools.partial()
  - functools.reduce(func, iter, [initial_value])
- 람다 표현식

#### functools 모듈
- functools 모듈은 고차원의 기능을 포함하고 있다. 
- 고차원의 함수는 하나 이상의 입력을 받아 새로운 함수를 리턴한다.
- 함수형 방식으로 작성된 프로그램 경우, 일부 매개 변수가 채워진 기존 함수의 변형이 필요한 경우가 있다. 파이썬 함수 f(a, b, c)가 있을 때, 함수 f(1, b, c)에 해당하는 새로운 함수 g(b, c)를 만들 수 있다.
- 이를 부분 함수 적용 이라고 한다.

#####  functools.partial()
- partial() 의 생성자는 (function, arg1, arg2, ..., kwarg1=value1, kwarg2=value2) 와 같은 인자를 취합니다. 결과 객체는 콜러블이므로, 채워진 인자로 function 을 실행하기 위해서는 결과 객체를 호출하면 됩니다.

In [1]:
import functools

def log(message, subsystem):
    """Write the contents of 'message' to the specified subsystem."""
    print('%s: %s' % (subsystem, message))

# 이름 그대로 부분적으로 subsystem 인자를 채운다??
server_log = functools.partial(log, subsystem='server')
# 결과 객체인 server_log 는 callable 이다.
server_log('Unable to open socket')

server: Unable to open socket


##### functools.reduce(func, iter, [initial_value])  
-  모든 이터러블 요소에 대해 누적 연산을 수행하므로 무한 이터러블에 적용할 수 없습니다. 
-  func 는 두 요소를 사용하여 하나의 값을 반환하는 함수이어야 합니다.

In [3]:
import operator, functools
# fr = operator.concat(operator.concat('A', 'BB'), 'C')
fr = functools.reduce(operator.concat, ['A', 'BB', 'C'])
print(fr)
fr = functools.reduce(operator.concat, [])
print(fr)

ABBC


TypeError: reduce() of empty sequence with no initial value

In [8]:
fr = functools.reduce(operator.mul, [1, 2, 3], 1)
print(fr)
fr = functools.reduce(operator.mul, [], 1)
print(fr)

# You can write:
product = 1
for i in [1, 2, 3]:
    product *= i
print(product)

6
1
6


##### itertools.accumulate(iterable, func=operator.add)
- functools.reduce와 차이를 파악하도록 한다. 결과 값을 반환하는 functools.reduce와는 달리 itertools.accumulate는 이터레이터를 반환한다.

In [12]:
import itertools

it = itertools.accumulate([1, 2, 3, 4, 5])
print(next(it))
print(next(it))
print(next(it))
print(next(it))

print()

it = itertools.accumulate([1, 2, 3, 4, 5], func=operator.mul)
print(next(it))
print(next(it))
print(next(it))
print(next(it))

1
3
6
10

1
2
6
24


#### 작은 함수와 람다 표현식
- 함수형 방식의 프로그램을 작성할 때, 서술자로 동작하거나 어떤 식으로든 요소를 결합하는 작은 함수가 필요할 것이다
- 파이썬 내장 함수나 적당한 모듈 함수가 있다면, 새로운 함수를 정의할 필요가 전혀 없다.
- lambda 는 여러 매개 변수와 이들 매개 변수를 결합하는 표현식을 취해 표현식의 값을 반환하는 익명의 함수를 만듭니다
  - lambda는 if 문을 가질 수 없고, try 문을 가질 수 없다.

In [13]:
# 파이썬에서는 함수도 일급 객체이기 때문에 가능한 일
# 함수로 리턴이 가능하고, 함수도 인자가 될 수 있다.
adder = lambda x, y: x + y
print_assign = lambda name, value: name + '=' + str(value)

def adder(x, y):
    return x + y

def print_assign(name, value):
    return name + '=' + str(value)

In [16]:
items = [(1, 2), (4, 5), (6, 7)]
total = sum(b for a, b in items)
print(total)

# 같은 의미 1, 이해하기 어렵다.
import functools
total = functools.reduce(lambda a, b: (0, a[1] + b[1]), items)[1]
print(total)

# 같은 의미 2, 이해하기 어렵다.
import functools
def combine(a, b):
    return 0, a[1] + b[1]

total = functools.reduce(combine, items)[1]
print(total)

# 같은 의미 3, 이해하기 어렵다.
total = 0
for a, b in items:
    total += b
print(total)


14
14
14
14


- 본 글에서는 lambda 표현식을 잘 권하지 않는다.
- functools.reduce() 를 사용하는 많은 경우, for 루프로 작성했을 때 더 명확합니다.
  
- Fredrik Lundh는 한때 lambda 사용법의 리팩토링을 위해 다음과 같은 규칙 집합을 제안했습니다:
  - 람다 함수를 작성하세요.
  - 람다가 하는 일에 관해 설명하는 글을 쓰세요.
  - 잠깐 설명을 검토하고 설명의 본질을 포착하는 이름을 생각해보세요.
  - 해당 이름을 사용하여 람다를 def 문으로 변환합니다.
  - 설명을 삭제하세요.

필자는 이 규칙을 정말 좋아하지만, 여러분은 이렇게 람다가 없는 방식이 더 나은지에 대해 동의하지 않을 수 있습니다.